## Because i am actually learning PyTorch and i want to be dual framework developer i made my tensorflow model translated into PyTorch. It is my first PyTorch model so there may be an errors :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  QuantileTransformer,  KBinsDiscretizer, PowerTransformer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import torch
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torchmetrics import AUROC
from scipy.stats import zscore


In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
X_test =  pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
train.pop('id')

y = train['claim']
train.pop('claim')
X_test.pop('id')
X=train
X['sum_na'] = X.isna().sum(axis=1)
X_test['sum_na'] = X_test.isna().sum(axis=1)
print(X.shape)

del train
gc.collect()

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp.fit_transform(X)
X_test = imp.transform(X_test)

X, X_off, y, y_off  = train_test_split(X,y,test_size=0.05,random_state=2021,stratify=y)
# z_scores = zscore(X)

# print(X.shape)
# abs_z_scores = np.abs(z_scores)
# filtered_entries = (abs_z_scores < 3.5).all(axis=1)
# y = y[filtered_entries]
# X = X[filtered_entries]
y = np.float32(y)
y_off = np.float32(y_off)
print(X.shape)

In [ ]:
class ClassModel(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.emb = nn.Embedding(106,18)
        self.fc = nn.Linear(119*18,20)#2380 for 20
        self.dropout = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(119,20)
        self.fc2 = nn.Linear(30,30)
        self.fc3 = nn.Linear(128*14,20)
        self.out = nn.Linear(60,1)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2,stride=2)
        self.pool2 = nn.MaxPool1d(kernel_size=2,stride=2)
        self.pool3 = nn.MaxPool1d(kernel_size=2,stride=2)
        torch.nn.init.xavier_normal_(self.out.weight)
        torch.nn.init.xavier_normal_(self.emb.weight)
        torch.nn.init.xavier_normal_(self.fc.weight)
        torch.nn.init.xavier_normal_(self.fc1.weight)
        torch.nn.init.xavier_normal_(self.fc2.weight)
        torch.nn.init.xavier_normal_(self.fc3.weight)
        torch.nn.init.xavier_normal_(self.conv1.weight)
        torch.nn.init.xavier_normal_(self.conv2.weight)
        torch.nn.init.xavier_normal_(self.conv3.weight)
    def forward(self, x_bin, x, x_conv):

        x_conv = F.relu(self.conv1(x_conv))
        x_conv = self.pool1(x_conv)
        x_conv = F.relu(self.conv2(x_conv))
        x_conv = self.pool2(x_conv)
        x_conv = F.relu(self.conv3(x_conv))
        x_conv = self.pool3(x_conv)

        x_conv = x_conv.view(-1,14*128)
        x_conv = F.silu(self.fc3(x_conv))
        x_bin = self.emb(x_bin)
        #x_bin = self.dropout(x_bin)
        x_bin = x_bin.view(-1,119*18)
        x_bin = F.silu(self.fc(x_bin))


        x = F.silu(self.fc1(x))
        #x = torch.add(torch.add(x_bin,x),x_conv)
        x = torch.cat((x,x_bin,x_conv),1)
        x = self.dropout2(x)
        #x = F.silu(self.fc2(x))
        #x = self.dropout2(x)
        x = torch.sigmoid(self.out(x))

        return x


### 8 - Fold Section

In [ ]:

def batch_gd(X, X_test, y, X_off,y_off,epochs):


    qt = QuantileTransformer(n_quantiles=106, output_distribution='normal')
    X = qt.fit_transform(X)
    X_test = qt.transform(X_test)
    X_off = qt.transform(X_off)
    bin_cat = KBinsDiscretizer(n_bins=106, encode='ordinal',strategy='uniform')
    X_bin = bin_cat.fit_transform(X)
    X_bin_off = bin_cat.transform(X_off)
    X_conv = X_bin.reshape(-1,1,119)/106
    X_conv_off = X_bin_off.reshape(-1,1,119)/106
    X_test_bin = bin_cat.transform(X_test)
    X_test_conv = X_test_bin.reshape(-1,1,119)/106

    gc.collect()
    kfold = StratifiedKFold(n_splits = 8, random_state=202109, shuffle=True)
    y_pred = torch.zeros(493474,1)
    metric = AUROC()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
       # device = 'cpu'
    print('Using {} device'.format(device))   
    y_pred = y_pred.to(device)
    X_test = torch.from_numpy(X_test.astype(np.float32))
    X_test_bin = torch.from_numpy(X_test_bin.astype(np.float32))
    X_test_conv = torch.from_numpy(X_test_conv.astype(np.float32))
    test_dataset=torch.utils.data.TensorDataset(X_test_bin.long(),X_test,X_test_conv)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)
    
    X_off = torch.from_numpy(X_off.astype(np.float32))
    X_bin_off = torch.from_numpy(X_bin_off.astype(np.float32))
    X_conv_off = torch.from_numpy(X_conv_off.astype(np.float32))
    y_off = torch.from_numpy(y_off.astype(np.float32).reshape(-1, 1))
    off_dataset=torch.utils.data.TensorDataset(X_bin_off.long(),X_off,X_conv_off, y_off)
    off_loader = torch.utils.data.DataLoader(off_dataset, batch_size=512, shuffle=False)
    for idx in kfold.split(X=X, y=y):
        best_metric = 0
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        xtrain_bin = X_bin[train_idx]
        xtrain_conv = X_conv[train_idx]
        ytrain = y[train_idx]
        xval = X[val_idx]
        xval_bin = X_bin[val_idx]
        xval_conv = X_conv[val_idx]
        yval = y[val_idx]
        
        ytrain = np.array(ytrain)
        yval = np.array(yval)
        
        xtrain = torch.from_numpy(xtrain.astype(np.float32))
        xval = torch.from_numpy(xval.astype(np.float32))
        ytrain = torch.from_numpy(ytrain.astype(np.float32).reshape(-1, 1))
        yval = torch.from_numpy(yval.astype(np.float32).reshape(-1, 1))

        xtrain_bin = torch.from_numpy(xtrain_bin.astype(np.float32))
        xval_bin = torch.from_numpy(xval_bin.astype(np.float32))
        xtrain_conv = torch.from_numpy(xtrain_conv.astype(np.float32))
        xval_conv = torch.from_numpy(xval_conv.astype(np.float32))
        
        train_dataset=torch.utils.data.TensorDataset(xtrain_bin.long(),xtrain,xtrain_conv,ytrain)
        val_dataset=torch.utils.data.TensorDataset(xval_bin.long(),xval,xval_conv,yval)

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False)
        
        model = ClassModel()
#         model.apply(weights_init)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.00023, eps=1e-08)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        model.to(device)
        gc.collect()
        for epoch in range(epochs):
            model.train()
            t0 = datetime.now()
            train_loss = []
            for inputs_bin, inputs, inputs_conv, targets in train_loader:
                inputs_bin, inputs, inputs_conv, targets = inputs_bin.to(device), inputs.to(device), inputs_conv.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs_bin,inputs,inputs_conv)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss.append(loss.item())
            train_loss = np.mean(train_loss) 

            model.eval()
            with torch.no_grad():
                val_loss = []
                for inputs_bin, inputs, inputs_conv, targets in val_loader:
                    inputs_bin, inputs, inputs_conv, targets = inputs_bin.to(device), inputs.to(device), inputs_conv.to(device), targets.to(device)
                    outputs = model(inputs_bin,inputs, inputs_conv)
                    loss = criterion(outputs, targets)
                    val_loss.append(loss.item())
                    auroc = metric(outputs,targets.int())

                val_loss = np.mean(val_loss)
                auroc = metric.compute()
                metric.reset()
                for inputs_bin, inputs, inputs_conv, targets in off_loader:
                    inputs_bin, inputs, inputs_conv, targets = inputs_bin.to(device), inputs.to(device), inputs_conv.to(device), targets.to(device)
                    outputs = model(inputs_bin,inputs, inputs_conv)
                    auroc_off = metric(outputs,targets.int())
                auroc_off = metric.compute()
                metric.reset()
                if auroc_off > best_metric:
                    best_metric = auroc_off
                    torch.save(model.state_dict(), '/kaggle/working/ckpt_pytorch')
        #         auc_roc = metric(outputs,targets.int()).compute()
                dt = datetime.now() - t0
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, \
              Val Loss: {val_loss:.4f} ,Best AUC {best_metric:.5f} , Off AUC {auroc_off:.4f} ,   Val AUC: {auroc:.4f} Duration: {dt}')
                auroc = metric.reset()
                auroc_off = metric.reset()
        with torch.no_grad():
            del model
            del inputs_bin
            del inputs
            del targets
            del xtrain
            del xtrain_bin
            del xval_bin
            del xval
            del ytrain
            del yval
            del train_dataset
            del val_dataset
            del train_loader
            del val_loader
            gc.collect()
            with torch.cuda.device('cuda:0'):
                torch.cuda.empty_cache()
            model = ClassModel()
            model.load_state_dict(torch.load('/kaggle/working/ckpt_pytorch'))
            model.to(device)
            preds = []
            model.eval()
            for inputs_bin, inputs, inputs_conv in test_loader:
                inputs_bin, inputs, inputs_conv = inputs_bin.to(device), inputs.to(device), inputs_conv.to(device)
                preds.append(model(inputs_bin, inputs, inputs_conv)/kfold.n_splits)
            y_pred += torch.cat(preds,0)

    return y_pred.cpu().numpy()




        

In [ ]:
y_pred=batch_gd(X, X_test, y, X_off,y_off, epochs=12)

In [ ]:
y_pred[:10]

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
sub.iloc[:,1]=y_pred
sub=sub.set_index('id')
sub.to_csv('baseline_pytorch_cv.csv')